In [1]:
require 'loadcaffe'
require 'image'
require 'nngraph'
prototxt = '../../Data/VGG_caffe/VGG_ILSVRC_16_layers_deploy.prototxt'
binary = '../../Data/VGG_caffe/VGG_ILSVRC_16_layers.caffemodel'

-- this will load the network and print it's structure
net = loadcaffe.load(prototxt, binary);


Successfully loaded ../../Data/VGG_caffe/VGG_ILSVRC_16_layers.caffemodel


conv1_1: 64 3 3 3
conv1_2: 64 64 3 3
conv2_1: 128 64 3 3
conv2_2: 128 128 3 3
conv3_1: 256 128 3 3


conv3_2: 256 256 3 3


conv3_3: 256 256 3 3


conv4_1: 512 256 3 3


...

conv5_2: 512 512 3 3


conv5_3: 512 512 3 3


fc6: 1 1 25088 4096


fc7: 1 1 4096 4096


fc8: 1 1 4096 1000


In [2]:
-- Loads the mapping from net outputs to human readable labels
function load_synset()
  local list = {}
  for line in io.lines'synset_words.txt' do
    table.insert(list, string.sub(line,11))
  end
  return list
end

function preprocess(img) 
-- 16 layer VGG expects a 3x224x224 sized image
  img = image.scale(img, 224, 224)
-- Directly obtained from the website
  local mean_pixel = torch.DoubleTensor({103.939, 116.779, 123.68})
-- Permuting from RBG to BGR
  local perm = torch.LongTensor{3, 2, 1}
-- Scaling the elements from 0:1 to 0:256
  img = img:index(1, perm):mul(256.0)
  mean_pixel = mean_pixel:view(3, 1, 1):expandAs(img)
-- Subtracting the mean
  img:add(-1, mean_pixel)
  return img
end

In [3]:
image_name = '../../Data/tiny-imagenet-200/test/images/test_4.JPEG'
-- loading thr image
im = image.load(image_name)
im = preprocess(im)


In [4]:
input = nn.Identity()()
conv1  = net.modules[1]
conv3  = net.modules[3]
conv6  = net.modules[6]
conv8  = net.modules[8]
conv11 = net.modules[11]
conv13 = net.modules[13]
conv15 = net.modules[15]
conv18 = net.modules[18]
conv20 = net.modules[20]
conv22 = net.modules[22]
conv25 = net.modules[25]
conv27 = net.modules[27]
conv29 = net.modules[27]
FC33   = net.modules[33]
FC36   = net.modules[36]
FC39   = net.modules[39]

level_0 = (conv1(input))
level_1 = conv3(nn.ReLU()(level_0))
level_2 = conv6(nn.SpatialMaxPooling(2,2,2,2)(nn.ReLU()(level_1)))
level_3 = conv8(nn.ReLU()(level_2))
level_4 = conv11( nn.SpatialMaxPooling(2,2,2,2)(nn.ReLU()(level_3)) )
level_5 = conv13(nn.ReLU()(level_4))
level_6 = conv15(nn.ReLU()(level_5))
level_7 = conv18(nn.SpatialMaxPooling(2,2,2,2)(nn.ReLU()(level_6)))
level_8 = conv20(nn.ReLU()(level_7))
level_9 = conv22(nn.ReLU()(level_8))
level_10 = conv25(nn.SpatialMaxPooling(2,2,2,2)(nn.ReLU()(level_9)))
level_11 = conv27(nn.ReLU()(level_10))
level_12 = conv29(nn.ReLU()(level_11))
level_13 = FC33(nn.View(-1)(nn.SpatialMaxPooling(2,2,2,2)(nn.ReLU()(level_12))))
level_14 = FC36(nn.Dropout(0.5)(nn.ReLU()(level_13)))
level_15 = FC39(nn.Dropout(0.5)(nn.ReLU()(level_14)))
output = nn.SoftMax()(level_15)

mlp = nn.gModule({input}, {output})
graph.dot(mlp.fg, 'MLP', 'VGGnet')



In [5]:
prob1,classes1 = mlp:forward(im):view(-1):sort(true) -- forward pass > resize to a single column > sort

In [6]:
-- Printing the class predicted by VGG net.
synset_words = load_synset()

for i=1,5 do
  print('predicted class '..tostring(i)..': ', synset_words[classes1[i]])
end

predicted class 1: 	scale, weighing machine	
predicted class 2: 	redbone	
predicted class 3: 	Rottweiler	
predicted class 4: 	vizsla, Hungarian pointer	
predicted class 5: 	shopping cart	


In [7]:
-- Preprocessing the image so and then feeding it to the VGG net.
prob,classes = net:forward(im):view(-1):sort(true) -- forward pass > resize to a single column > sort

In [8]:
-- Printing the class predicted by VGG net.
synset_words = load_synset()

for i=1,5 do
  print('predicted class '..tostring(i)..': ', synset_words[classes[i]])
end

predicted class 1: 	albatross, mollymawk	
predicted class 2: 	killer whale, killer, orca, grampus, sea wolf, Orcinus orca	
predicted class 3: 	red-breasted merganser, Mergus serrator	
predicted class 4: 	king penguin, Aptenodytes patagonica	
predicted class 5: 	grey whale, gray whale, devilfish, Eschrichtius gibbosus, Eschrichtius robustus	


In [23]:
i = 39
layer_mlp = mlp.modules[i+1].output
layer_1 = net.modules[i].output
print(mlp.modules[i+1])
print(net.modules[i])
print("done")

nn.Linear(4096 -> 1000)
{
  gradBias : DoubleTensor - size: 1000
  weight : DoubleTensor - size: 1000x4096
  gradWeight : DoubleTensor - size: 1000x4096
  gradInput : DoubleTensor - empty
  name : fc8
  bias : DoubleTensor - size: 1000
  output : DoubleTensor - size: 1000
}
nn.Linear(4096 -> 1000)
{
  gradBias : DoubleTensor - size: 1000
  weight : DoubleTensor - size: 1000x4096
  gradWeight : DoubleTensor - size: 1000x4096
  gradInput : DoubleTensor - empty
  name : fc8
  bias : DoubleTensor - size: 1000
  output : DoubleTensor - size: 1000
}
done	


In [24]:
print(layer_mlp - layer_1)

 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 

In [ ]:

layer_mlp = mlp.modules[2].output
rescaled_layer_mlp = image.scale(layer_mlp,224,224,'simple');
itorch.image(rescaled_layer_mlp)

In [ ]:

layer_1 = net.modules[2].output
rescaled_layer_1 = image.scale(layer_1,224,224,'simple');
itorch.image(rescaled_layer_1)

In [ ]:
-- Creating the hypercolumns
layer_1 = net.modules[5].output;
hyper_columns = image.scale(layer_1,224,224,'simple');
layer_nums = {9,16,23,30}
for i=1,3 do
    layer = net.modules[layer_nums[i]].output
    hyper_columns = torch.cat(hyper_columns,image.scale(layer,224,224,'simple'),1)
end
    

In [ ]:
--itorch.image(image.scale(hyper_columns,224,224))
itorch.image(hyper_columns)


In [ ]:
require("Utils")
image_name = '../../Data/tiny-imagenet-200/test/images/test_4.JPEG'
-- loading thr image
im = image.load(image_name)
itorch.image(im)
-- Calling the rgb2gray function
grayim = rgb2gray(im)
itorch.image(grayim)
-- Calling the gray2rbg function

grayim3 = gray2rgb(grayim)
itorch.image(grayim3)

In [ ]:
hc = {hyper_columns,hyper_columns}
hc_file_name = '../../Data/tiny-imagenet-200/test/hc/test_4.t7'
torch.save(hc_file_name, hc, "binary");

In [ ]:
hc_loaded = torch.load(hc_file_name)

In [ ]:
itorch.image(hyper_columns)